# Data analysis

## Pandas 

In [ ]:
import pandas as pd

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"

df= pd.read_csv(url, header=None)

df.head(n) muestra las primera n columnas del archivo (default 5)

In [ ]:
df.head(10)

df.tail(n) muestra las ultimas n columnas del archivo

In [ ]:
df.tail(10)

Para escribir el archivo en nuestro dominio podemos agregar un path

In [ ]:
path="/home/kevindroide/NDS/DataScience/TESE2022/automovile.csv"
#df.to_csv(path)

 Pandas ofrece lecturas y exportaciones de archivos csv, json, Excel, sql

    pd.read_csv() ----- df.to_csv()
    pd.read_json() ---- df.to_json()
    pd.read_excel() ---- df.to_excel()
    pd.read_sql() ---- df.to_sql()

El DataSet no tiene argumentos en header, por lo que podemos agregarlos:

In [ ]:
headers = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]
print("Headers:\n", headers)

In [ ]:
df.columns = headers
df.head(10)

In [ ]:
df.describe()

df.describe() ignora las columnas vacias, para incluir todas agregamos parametros
df.describe(include="all")

In [ ]:
df.describe(include="all")

Se agregan tres parametros más a la estadística:


unique corresponde a la cantidad de objetos distintos en la columna

top corresponde al objeto más frecuente en la columna

freq corresponde a la frecuencia en la que el objeto top aparece

(NaN significa Not a Nomber)

Otro método para revisar el DataSet es df.info()

In [ ]:
df.info()

Para deshacernos de los valores nulos podemoa usar df.dropna()

df.drop() hace lo mismo pero con columnas enteras

El metodo por sí solo no genera cambios en el DataFrame, para ello se agrega el parámetro inplace=True

In [ ]:
df.dropna(subset=["price"], axis=0)
#en axis, 0 representa la fila/row y 1 representa la columna

Para obtener el DataType del DataFrame podemos usar df.dtype()

In [ ]:
print(df.dtypes)

Podemos hace tambien de descripcion de headers específicos como parametros en un arreglo del DataFrame:

In [ ]:
df[["length", "compression-ratio"]].describe()

## Remplazar valores
Para esto se usa el método df.replace(valor_a_remplazar, nuevo_valor)

Es comun que si un valor está extraviado se use el valor promedio de la columna, entonces calculamos el promedio de la "normalized-losses"

np.nan() retorna los valores NaN

Reemplazamos los valores perdidos de ? para poder hacer un promedio

El metodo replace() da una salida al reemplazamiento pero no modifica el Dataframe, por lo tanto, se hace una asignación

In [ ]:
import numpy as np

df["normalized-losses"] = df["normalized-losses"].replace("?",np.NaN).astype(float)

In [ ]:
df["normalized-losses"]

In [ ]:
#obtenemos el promedio y remplazamos
prom = df["normalized-losses"].mean().astype(int)
print(prom)

In [ ]:
df["normalized-losses"]= df["normalized-losses"].replace(np.NaN,prom)

In [ ]:
df.head()

Ahora puede verse que los valores nulos "?" fueron reemplazados por el valor promedio 122

### Para identificar el tipo de dato:
    df.dtype()
### Para convertir a un tipo de dato
    df.astype()

## Normalización
Tres formas de normalización      

# i)    $x_{nor}$ = ${x_i}\over{x_{max}} $  (Simple Feature scaling)

# ii) $ x_{nor}$ = ${x_i - x_{min}} \over {x_{max} - x_{min}} $ (Min-Max)

# iii) $x_{nor}$ = ${x_i -\mu} \over {\sigma}$ (Z-score)

En Python podemos escribir:

>df["feature"] = df["feature"]/df["feature"].max()
    
>df["feature"] = (df["feature"] - df["feature"].min())/(df["feature"].max - df["feature"].min())

>df["feature"] = (df["feature"] - df["feature"].mean())/df["feature"].std()

##  Binning
Binning es cuando se categorizan valores de una variable en acotaciones
   
   i.e. edad: [0,6],[7,13],[14,18],[19,30]...

Podemos usar el df anterior para categorizar precios como: Low, Normal, High

Para ello creamos particiones de los precios usando Numpy

In [ ]:

#"price" es de tipo object, por lo que reemplazaremos los valores

df["price"] = df["price"].replace("?",np.NaN).astype(float)
df["price"].dropna(axis=0)
df["price"] = df["price"].astype(float)

In [ ]:
bins = np.linspace(min(df["price"]), max(df["price"]), 4)

In [ ]:
price_bins = ["Low","Medium","High"]

Podemos usar el metodo de pandas pd.cut() para hacer el arreglo de los bins

In [ ]:
df["price-binned"] =  pd.cut(df["price"],bins,labels=price_bins,include_lowest=True)

In [ ]:
df["price-binned"].hist()

# Variables categoricas a numericas
Eventualmente encontramos variables categoricas, como el tipo de combustible, para hacer conteo de cada una de las categorias podemos usar
>pd.get_dummies() 

Este proceso se conoce como 
### One-Hot encoding

In [ ]:
pd.get_dummies(df["fuel-type"])

# Herramientas de visualización
## matplotlib

Es una librería para la creación de graficas

Podemos empezar con un histograma de los precios de los autos

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df["price"], bins=10, color="red")

Podemos agregar notaciones a nuestras graficas para un mejor entendimiento de lo que se grafica

In [ ]:
plt.hist(df["price"], bins=15, color="green", alpha=0.5)
plt.title("Distribución de precios de autos en USD")
plt.xlabel("Precio en USD")
plt.ylabel("Frecuencia")

Podemos también agrupar y acomodar datos de interés, como el tipo de cuerpo.

También podemos hacer calculos sobre los grupos extraidos.

In [ ]:
df_body = df.groupby("body-style")["price"].mean()
df_body.head()

Así, también podemos graficar sobre nuestras nuevas agrupaciones.

Usaremos una grafica de barras para visualizar los precios promedio por tipo de cuerpo del auto

In [ ]:
df_body.plot(kind="bar", width=0.5, color="green", title="Precios promedio por tipo de cuerpo", xlabel="Tipo de cuerpo", ylabel="Precios promedio")

In [ ]:
df_body["frecuency"] = df["body-style"].value_counts().sort_index()
df_body["frecuency"].plot(kind="pie", figsize=(5, 6), autopct='%1.1f%%', title="Body styles in dataframe")

In [ ]:
df["horsepower"] = df["horsepower"].replace("?",np.NaN).astype(float)
df["horsepower"].dropna(axis=0, inplace=True)
df["peak-rpm"] = df["peak-rpm"].replace("?",np.NaN).astype(float)
df["peak-rpm"].dropna(axis=0, inplace=True)

In [ ]:
norm_price = (df["price"] - df["price"].min()) / (df["price"].max() - df["price"].min())
#norm_price = (df["price"] - df["price"].mean())/(df["price"].std())
df.sort_values("horsepower").plot(
        kind="scatter",
        x="horsepower",
        y="peak-rpm",
        alpha=0.3,
        s=norm_price*3000+10,
        xlim=(25,300),
        figsize=(10,10))

In [ ]:
df.groupby("drive-wheels")["price"].mean().plot(kind="bar", width=0.5, color="blue")


# Exploratory Data Analysis
### Estadística descriptiva

Podemos usar el método df.describe() para obtener infromación estadística del DataSet, sin embargo, ésta función ignora los elementos NaN.

Sin embargo, podemos describir datos categóricos usando el método value_counts()

In [ ]:
df["fuel-type"].value_counts()

## Boxplots
### Diagrama de cajas
Con Boxplots podemos hacer comparaciones en la distribución de la información

Hagamos otro DataFrame con los datos del tipo de tracción

In [ ]:
drive_wheels_counts= df["drive-wheels"].value_counts()
drive_wheels_counts.rename({"drive-wheels":"value_counts"},inplace=True)
drive_wheels_counts.index.name="drive-wheels"

In [ ]:
drive_wheels_counts

In [ ]:
import matplotlib as mpl
import seaborn as sns

sns.boxplot(x="drive-wheels", y="price", data=df)

En un grafico de cajas:

    -La linea del centro de la caja representa la mediana. Una mitad de los datos está por debajo de este valor, y la otra por encima. 
    
    -Los extremos de arriba y abajo de la caja indican los cuantiles, o percentiles, 25 y 75. Estos dos cuantiles también se conocen como cuartiles, porque separan cuartos (25 %) de los datos. La longitud de la caja es la diferencia entre estos dos percentiles y se conoce como rango intercuartílico (IQR).

    -Las líneas que se extienden desde la caja se llaman bigotes. Los bigotes representan la varianza esperada de los datos.

    -Si hay datos que queden por encima o por debajo de los extremos de los bigotes, se los representa con puntos. Estos puntos se conocen como valores atípicos. Un valor atípico es el que supera la varianza esperada. Merece la pena revisar estos puntos de datos para aclarar si son atípicos o erróneos.
        


Así podemos ver que existe una variación en el precio más grande entre los sistemas de llantas traseras que entre los sistemas de cuatro llantas

## Scatter Plot
Ayudan a detectar si existe correlación entre los datos, cuenta con dos variables:

    Predictor/Independent (La que nos interesa si tiene correlación)
    Target/dependent (La que intentamos predecir

In [ ]:
y=df["engine-size"]
x=df["price"]

plt.scatter(x,y)
plt.title("Relacíon Tamaño del motor VS Precio")
plt.ylabel("Tamaño del motor")
plt.xlabel("Precio")
plt.show()

Así, podemos ver que a mayor precio, existe una tendencia a mayor tamaño del motor

## Grouping
Pandas contiene el método df.Groupby(), éste puede aplicarse a variables categóricas.
Ésta crea grupos de datos en subsets incluyendo las distintas categorias de la variable, que puede ser una o varias.

Podemo así extraer columnas de interes para analizar si el tipo de sistema de llantas tiene relación con el precio:

In [ ]:
df_wheels = df[["drive-wheels", "body-style",  "price"]]

In [ ]:
df_grupo =  df_wheels.groupby(["drive-wheels","body-style"], as_index=False).mean()
df_grupo

Esta tabla no es tan fácil de leer a simple vista, por lo que podemos usar el método df.pivot()
para poder reordenar el indice y las columnas de interés:

In [ ]:
df_pivot = df_grupo.pivot(index="drive-wheels", columns="body-style")
df_pivot

## Heatmap

Heatmap es un método de matplotlib que toma una tabla rectangular y les asigna un color vasado en el valor de cada elemento. Así pueden analizarse facilmente un conjunto de variables a la vez.

In [ ]:
#plt.pcolor(df_grupo["body-style"], df_grupo["drive-wheels"], np.array(df_grupo["price"]), cmap="RdBu")
plt.pcolor(df_pivot, cmap="RdBu")
plt.colorbar()
plt.yticks(np.arange(0.5, len(df_pivot.index), 1), df_pivot.index)
plt.xticks(np.arange(0.5, len(df_pivot.columns), 1), df_pivot.columns.get_level_values(1))
plt.show(6,5)


## ANOVA
Analysis of Variance
El método ANOVA encuentra la correlación entre distintos grupos de una variable categórica. Este método retorna dos elementos:

## $ F_{statistic} $ = $ varianza_{grupo} \over varianza_{grupos} $

    F-statistic score: Variación entre el promedio del grupo divividido por la varianza

        -Un valor grande de F implica una gran varianza entre la categoría y la variable objetivo
        
        -Un valor pequeño de F implica poca varianza entre la variable categórica y la variable objetivo
        
#### Entre mayor sea el valor de F-statistic, mayor diferencia habrá entre los grupos 

    
    p-value: Valor de confianza, el valor de esto sintetiza la reelevancia de la estadística

#### $ {p} < 0.05 $ Podemos concluir que hay una diferencia estadística significativa entre los promedios de cada grupo

#### $ {p} >> 0.05 $ Sigifica que no existen suficientes datos para concluir la diferencia significativa entre los promedios de cada grupo
    
Para éste método es necesario la librería SciPy.stats

In [ ]:
import scipy.stats as stats

df_anova = df[["make", "price"]]
grouped_anova = df_anova.groupby(["make"])

In [ ]:
anova_results = stats.f_oneway(grouped_anova.get_group("honda")["price"], grouped_anova.get_group("subaru")["price"])
anova_results

#### 'statistic' representa el valor de F, que en este caso es menor a 1, entonces no hay mucha diferencia entre los precios de ambos grupos

#### Pero p >> 0.05, entonces no se puede comprobar la hipótesis de que los promedios de los grupos son similares.

Comparando ahora contra la marca "jaguar":

In [ ]:
anova_results = stats.f_oneway(grouped_anova.get_group("honda")["price"], grouped_anova.get_group("jaguar")["price"])
anova_results

#### Podemos ahora ver un valor de F muy grande y un valor de p muy pequeño, por lo que se concluye que existe gran varianza entre lo precios de Honda y Jaguar

# Correlation

Mide en qué forma distintas variables son independientes


In [ ]:
sns.regplot(x="engine-size", y="price", data=df, color="green")
plt.style.use("ggplot")
plt.ylim(0,)

Podemos ver una tendencia en una pendiente positiva.

Hagamos otro análisis entre galones por litro y el precio:

In [ ]:
sns.regplot(x="highway-mpg", y="price", data=df)
plt.ylim(0,)

Ahora podemos ver una pendiente negativa: entre más millas por galón consume un auto, menor tiende a ser su precio.

Así podemos decir que existe una relación lineal negativa entre mpg y el precio

Podemos tambien descartar otras relaciones que no muestren una tendencia, por ejemplo rpm:

In [ ]:
# "peak-rpm" es de tipo object, por lo que lo convertimos a tipo int
#df["peak-rpm"] = df["peak-rpm"].replace("?",0).astype(int)
#df["peak-rpm"] = df["peak-rpm"].astype(int)

#Ajustamos tambien los 0 al promedio para no tener incongruencias
rpm_prom = df["peak-rpm"].mean().astype(int)
df["peak-rpm"] = df["peak-rpm"].replace(np.NaN,rpm_prom).astype(int)


In [ ]:
sns.regplot(x="peak-rpm", y="price", data=df)
plt.ylim(0,)

Podemos ver que el valor de RPM no tiene una correlación evidente, por lo que los datos de RPM no son efectivos para predecir precios

## Pearson Correlation
Mide una magnitud de correlación en dos prámetros:
    
    Coeficiente de correlación:
-Valores cercanos a 1 indican una grande correlación positiva

-Valores cercanos a -1 indican una grande correlación negativa

-Valores cercanos a 0 indican que no hay correlación entre la variabes

    P-value:
Mide qué tan certero es el coeficiente de correlación anterior

P-value < 0.001 indican gran certeza

P-value < 0.05 indican certeza moderada

P-value < 0.1 indican certeza débil

P-value > 0.1 indican que no hay certeza en el resultado



### Así podemos decir que una correlación es fuerte si el coeficiente es cercano a 1 o a -1 y que el P-value es menor a 0.001

In [ ]:
# "horsepowe" es de tipo object, por lo que lo convertimos a tipo int
#df["horsepower"] = df["horsepower"].replace("?",0).astype(int)
#df["horsepower"] = df["horsepower"].astype(int)

#Ajustamos tambien los 0 al promedio para no tener incongruencias
rpm_prom = df["horsepower"].mean().astype(int)
df["horsepower"] = df["horsepower"].replace(np.NaN,rpm_prom).astype(int)


In [ ]:
df[df["price"].isnull()]


In [ ]:
df["price"] = df["price"].apply(pd.to_numeric, errors = 'coerce')
df.dropna(how='any', inplace=True)
df[df["price"].isnull()]


In [ ]:
pearson_coef, p_value = stats.pearsonr(df["horsepower"], df["price"])
print("Pearson Coef: ", pearson_coef, " P-value: ", p_value)

Podemos ver que el Coeficiente de Pearson es 0.1, por lo que es cercano a 1, y el valor de p es muy muy pequeño, por lo que hay certeza en el resultado

Puede entonces decirse que existe gran correlación positiva entre el "horsepower" y el precio

Podemos también usar el método de pandas df.corr() para hacer una menos descriptiva pero más general de nuestro Dataframe:

In [ ]:
df.corr()

In [ ]:
df[["price","horsepower", "engine-size", "compression-ratio", "city-mpg","peak-rpm"]].corr()

Así, en la primera fila podemos encontrar las correlaciones entre el precio y los otros atributos.

Podemos decir con ésto que para el precio hay más reelevancia en el tamaño del motor que en las rpm

In [ ]:
corr = df.corr()
plt.figure(figsize=(15,15))
sns.heatmap(corr, xticklabels=corr.columns, 
            yticklabels=corr.columns, 
            annot=True,
            cmap=sns.diverging_palette(220, 30, as_cmap=True))


# Model Development

### Simple Linear Regression

La regresión lineal simple es un método para comprender la relación entre dos variables:
    
    Predictor/independiente: x
    Target/dependiente: y
    
Deseamos encontrar una relación lineal entre ambas variables de la forma:

# $y$ = $b_0 + b_1 x $

El parámetro $b_0$ es la insersección, $b_1$ es la pendiente de la recta.

Utilizaremos linear_model del paquete scikit-learn
    


In [ ]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()

Definimos las variables independiente y dependiente:

In [ ]:
X = df[["highway-mpg"]]
Y = df["price"]

El metodo lm.fit() para ajustar los parámetros del modelo y encontrar los parámetros $b_0, b_1$

Podemos obtener la predicción con el método lm.predict(), cuya salida es un arreglo

In [ ]:
lm.fit(X,Y)
Yhat=lm.predict(X)
print(X.size)
print(Y.size)
print(Yhat.size)

Podemos encontrar la intersección $b_0$ usando lm.intercept_, y obtenemos $b_1$ usando lm.coef_

In [ ]:
print("b_0 = ", lm.intercept_)
print("b_1 = ",lm.coef_)

Por lo tanto, la relación lineal quedará representada como

    Precio = 38423.30 -(821.73)*highway-mpg

### Multiple Linear Regression

Éste método permite encontrar la relación entre:

    Una variable Target: y
    
    Dos o más variables Predictors: x
    
Si tuvieramos cuatro varirables predictorias, tendríamos una ecuación tal que:

# $\hat{Y}$ = $b_0 + b_1 x_1 + b_2 x_2 + b_3 x_3 + b_4 x_4 $

Para extraer las variables que queremos predecir las guardamos en una variable:

In [ ]:
Z = df[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']]
lm.fit(Z,df["price"])


In [ ]:
print(lm.intercept_)
print(lm.coef_)

#### lm.intercept_ representa la intersección, es decir, el término independiente de la relación lineal, mientras lm.coef_ representa la pendiente

### Regression plot

Nos da un estimado de la relación de dos variables, la relevancia de la correlación y la direcciónn de la la relación. 
Para visualizarlo usaremos regplot() de seaborn

In [ ]:
sns.regplot(x="highway-mpg", y="price", data=df)
plt.ylim(0,)

### Residual plot

El residual plot representa el error entre los valores reales. El residuo representa la distancia entre el valor predecido Yhat y el valor correcto Y

In [ ]:
sns.residplot(x="highway-mpg", y="price", data=df)

### Distrubition plot

Podemos hacer una gráfica de la dristribución entre los valores predecidos con los valores reales

In [ ]:
plt.figure(figsize=(10,10))
ax1 = sns.distplot(df["price"], hist=False, color="r", label="Valor real")
sns.distplot(Yhat, hist=False, color="b", label="Prediccion", ax=ax1)
plt.title("Valores reales contra predicción")
plt.xlabel("Precio")
plt.ylabel("Proporción del auto")


### Polonomial regression

Cuando el modelo no puede predecirse con un modelo lineal podemos utilizar una regresión polinómica. Es práctica para adaptar modelos con curvaturas.

    Regresión polinómica de orden n:
$\hat{Y}$ = $ b_0 + b_1 x_1 + b_2 {x_2}^2 + ... + b_n {x_n}^n $

En python podemos usar numpy para ajustar modelos polinómicos usando np.polyfit() y np.poly1d() para mostrarlo

#### Definiremos una función para poder graficar polinomios

In [ ]:
def PlotPolly(model, independent_variable, dependent_variabble, Name):
    x_new = np.linspace(15, 55, 100)
    y_new = model(x_new)

    plt.plot(independent_variable, dependent_variabble, '.', x_new, y_new, '-')
    plt.title('Polynomial Fit with Matplotlib for Price ~ Length')
    ax = plt.gca()
    ax.set_facecolor((0.898, 0.898, 0.898))
    fig = plt.gcf()
    plt.xlabel(Name)
    plt.ylabel('Price of Cars')

    plt.show()
    plt.close()

In [ ]:
x = df["highway-mpg"]
y = df["price"]

In [ ]:
f = np.polyfit(x,y,3)
p = np.poly1d(f)
print(p)

In [ ]:
PlotPolly(p, x, y, "Highway-mpg")

Podemos usar "preprocessing" en sci-kit-learn para crear un objeto polinómico.

El constructor toma el grado del polinomio como parámetro para después poder transformarlo en un objeto con fit_transform()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

pr= PolynomialFeatures(degree=2, include_bias=False)

x_polly = pr.fit_transform(df[["horsepower", "curb-weight"]])

Podemos normalizar cada atributo simultaneamente:

In [ ]:
from sklearn.preprocessing import StandardScaler

SCALE = StandardScaler()

SCALE.fit(df[["horsepower", "highway-mpg"]])

x_scale = SCALE.transform(df[["horsepower", "highway-mpg"]])

Podemos simplificar éstos códigos usando Pipelines, que sigue los pasos:

Normalización ---> Transformación Polinómica ---> Regresión Lineal

In [ ]:
from sklearn.pipeline import Pipeline

Input = [("scale", StandardScaler()), ("polynomial", PolynomialFeatures(degree=3)), ("model", LinearRegression())]

pipe = Pipeline(Input)

pipe

In [ ]:
pipe.fit(Z,y)

Crearemos una funcion para poder graficar nuestra regresión polinómica:

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual

def PollyPlot(xtrain, xtest, y_train, y_test, lr,poly_transform):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))
    
    
    #training data 
    #testing data 
    #lr:  linear regression object 
    #poly_transform:  polynomial transformation object 
 
    xmax=max([xtrain.values.max(), xtest.values.max()])

    xmin=min([xtrain.values.min(), xtest.values.min()])

    x=np.arange(xmin, xmax, 0.1)


    plt.plot(xtrain, y_train, 'ro', label='Datos de entrenamiento')
    plt.plot(xtest, y_test, 'go', label='Datos de prueba')
    plt.plot(x, lr.predict(poly_transform.fit_transform(x.reshape(-1, 1))), label='Función de regresión', linewidth=2, color='blue',)
    plt.ylim([-10000, 60000])
    plt.ylabel('Price')
    plt.legend()
    
    return plt

def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False, color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Precio [USD]')
    plt.ylabel('Proportion of Cars')

def f(order, test_data):
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=test_data, random_state=0)
    pr = PolynomialFeatures(degree=order)
    x_train_pr = pr.fit_transform(x_train[['horsepower']])
    x_test_pr = pr.fit_transform(x_test[['horsepower']])
    poly = LinearRegression()
    poly.fit(x_train_pr,y_train)
    PollyPlot(x_train[['horsepower']], x_test[['horsepower']], y_train,y_test, poly, pr)
    

### Evaluación del modelo

Son formas para evaluar qué tan preciso es el modelo que tenemos comparado con los valores reales. Dos métodos son importantes:

    Mean Squared Error (MSE): Error cuadrático medio
Debemos encontrar la diferencia entre los valores reales (y) y los valores predecidos (Yhat) y elevarla al cuadrado. Posteriormente se obtiene el promedio del cuadrado de los errores.
    
    R-squared (R^2):
También llamado coeficiente de determinación, mide qué tan cercanos son los valores reales contra los predecidos. Representa el porcentaje de la variación del valor real que puede ser explicado por el modelo
    
# $R^2$ = $ ({1} - {{ MSE_{regression-line}} \over {MSE_{average-data}}}) $
    
    
Para obtener el MSE podemos usar el método mean_squared_error(valor real, valor predecido): 

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(df["price"], Yhat)

El valor de $R^2$ lo podemos obtener usando el método lm.score()

In [ ]:
X=df[["highway-mpg"]]
Y=df["price"]

lm.fit(X,Y)

lm.score(X,Y)

Esto dice que el modelo lineal predice el 49.65% de la variación del precio

# Prediction and  decision making

Determinaremos si un modelo es de fiar, para esto podemos utilizar visualización, métodos numéricos y comparación entre distintos modelos.

Primero entrenamos el modelo con lm.fit() y después agregamos como parámetro el valor que queremos determinar en lm.predict()


In [ ]:
lm.fit(X,Y)
lm.predict(X)[30] #Corresponde a una predicción del precio de un auto con 30 highway-mpg

El primer método para visualizar y hacer un análisis del model es un RegressionPlot, pues puede apreciarse de una forma más clara la correspondencia de los valores.

En un Resicual plot podemos aún confirmar el comportamiento de las gráficas: si podemos apreciar una curva, se sugiere un comportamiento no-lineal.

Los Distribution plot son un buen método para evaluar múltiples variables.

El MSE es intuitivo para determinar si el modelo es bueno o no, así como lo es también $R^2$


# Evaluación y refinación del modelo

Anteriormente hemos evaluado nuestros modelos en base a la información con la que fue entrenado. Sin embargo aún no hemos evaluado un modelo para predecir nueva información.

Separar la información entre training/testing sets es algo importante para la evaluación del modelo. Usualmente, la gran mayoría de la información es usada para entrenar el modelo y sólo una pequeña parte es utilizada para testing.

Por ejemplo, podemos dividir el 70% de un dataset para entrenar el modelo y el otro 30% para probarlo. Posteriormente construimos el modelo usando training data y usamos la testing data para medir la eficiencia del modelo. Una vez terminado el testing, podemos usar toda la información para darle un mejor desempeño.

Una función popular para divivir un DataSet es la función train_test_split() en el paquete scikit-learn, que divide aleatoriamente la información que necesitamos. Por ejemplo:

       x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=03, random_state=0)
Donde x_data corresponde a la variable independiente, como los features del sistema, y y_data es el objetivo a predecir, como el precio. x_train, y_train corresponde al dataset para entrenar el modelo y x_test, y_test son para evaluar el modelo.
test_size corresponde al percentaje del dataset para la evaluación. random_state es una semilla para la separación aleatoria.

Generalization error es una medida para qué tan bien la información usada ayuda a predecir nueva información.

Cross Validation es una métrica de evaluación de la información, tanto de la de entrenamiento como la de evaluación. Cross validation crea distintas particiones del dataframe y permuta el testing data para deducir si la información puede producir un buen desempeño:

        scores = cross_val_score(lr, x_data, y_data, cv=3)
lr representa "linear regression" como el tipo de modelo y cv corresponde a la cantidad de particiones que se harán. El retorno se expresa como un arreglo, sin embargo podemo usar no.mean().

    yhat = cross_val_predict(lr2e, x_data, y_data, cv=3)
Tiene la misma sintaxis pero no devuelve una evaluación del modelo, sino la misma predicción del modelo que medimos.

In [ ]:
from sklearn.model_selection import train_test_split

y_data = df["price"]
x_data = df.drop("price", axis=1)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.15, random_state=1)

print("Número de datos en test: ", x_test.shape[0])
print("Número de datos en train: ", x_train.shape[0])

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

lre=LinearRegression()

Rcross = cross_val_score(lre, x_data[['horsepower']], y_data, cv=4)
Rcross

In [ ]:
print("El promedio de los 'folds'es: ", Rcross.mean(), "y la desviación es: " , Rcross.std())

In [ ]:
from sklearn.model_selection import cross_val_predict

yhat = cross_val_predict(lre, x_data[["horsepower"]], y_data, cv=4)

yhat[:5]

### Overfitting, Underfitting and Model Selection

Asumiremos que los training points vienen de un función polinómica de la forma

$y(x)+ ruido$

El objetivo de la seleccion de un modelo es encontrar el orden del polinomio que mejor predice la función. En el caso anteior de una función polinómica, un modelo lineal resulta vago para expresar correctamente la función

Llamaremos Undefitting cuando un modelo es demasiado simple como para describir propiamente un sistema.

Al aumentar el polinomio del modelo lineal podemos hacer una aproximación mas certera del modelo, sin embargo, sucede también que un polinomio de grado muy alto puede predecir exactamente los puntos de prueba, pero falla en describir la función.

Llamamos entonces Overfitting cuando el modelo se ajusta más a los puntos de ruido que a la función que buscamos, ésto es común en las regiones con pocos data points.

Una buena forma de ubicar las regiiones de Underfitting y Overfitting es con los valores $R^2$. Podemos usar el siguiente código para hacer reconocimiento de los valores de $R^2$ según el orden del polinomio: 

In [ ]:
lr=LinearRegression()

#lr.fit(x_train[["horsepower"]], y_train)

Rsqu_test = []
order = [1,2,3,4,5]
for n in order:
    pr=PolynomialFeatures(degree=n)
    x_train_pr = pr.fit_transform(x_train[["curb-weight"]])
    x_test_pr = pr.fit_transform(x_test[["curb-weight"]])
    lr.fit(x_train_pr, y_train)
    Rsqu_test.append(lr.score(x_test_pr, y_test))
#PollyPlot(x_train[["curb-weight"]], x_test[["curb-weight"]], y_train, y_test, lr, pr, n)
interact(f, order=(1,10,1), test_data=(0.05,0.95,0.05))
for n in order:
    print('Polynomial degree:' , n, 'R-squared:', Rsqu_test[n-1])

print('Max score:', max(Rsqu_test), 'with polynomial degree:', order[Rsqu_test.index(max(Rsqu_test))])



In [ ]:
plt.plot(order, Rsqu_test)
plt.xlabel('order')
plt.ylabel('R^2')
plt.title('R^2 por grado polinomial')
plt.text(2.2, 0.835, 'Maximum R^2 ')  


### Ridge regression

Éste método nos previene de hacer Overfitting. Éste método crea una nueva variable "Alpha", la cual regula los coeficientes de nuestro polinomio. Éste es un parámetro que escogemos antes de entrenar nuestro modelo.

Para hacer una predicción usando ridge regression podemos usar un codigo como el siguiente:

In [ ]:
from sklearn.linear_model import Ridge

pr=PolynomialFeatures(degree=2)
x_train_pr=pr.fit_transform(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']])
x_test_pr=pr.fit_transform(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']])

RigeModel = Ridge(alpha=1)

RigeModel.fit(x_train_pr,y_train)

yhat =  RigeModel.predict(x_test_pr)

print('predicted:', yhat[0:4])
print('test set :', y_test[0:4].values)



Podemos crear un loop para encontrar el valor de Alpha más eficiente para el modelo

In [ ]:
from tqdm import tqdm

Rsqu_test = []
Rsqu_train = []
dummy1 = []
Alpha = 10 * np.array(range(0,1000))
pbar = tqdm(Alpha)

for alpha in pbar:
    RigeModel = Ridge(alpha=alpha) 
    RigeModel.fit(x_train_pr, y_train)
    test_score, train_score = RigeModel.score(x_test_pr, y_test), RigeModel.score(x_train_pr, y_train)
    
    pbar.set_postfix({"Test Score": test_score, "Train Score": train_score})

    Rsqu_test.append(test_score)
    Rsqu_train.append(train_score)

Podemos graficar el comportamiento de estos valores

In [ ]:
width = 12
height = 10
plt.figure(figsize=(width, height))

plt.plot(Alpha,Rsqu_test, 'b',label='Test')
plt.plot(Alpha,Rsqu_train, 'r', label='Train')
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.legend()

### Grid Search

Éste método nos permite hacer un análisis de múltiples parámetros en pocas lineas de código.
Parámetros como "Alpha" no son parte del proceso de entrenamiento del modelo, por lo que se les refiere como hyperparámetros.

Grid search toma los modelos u objetos de los que se tenga interés y los evalua bajo distintos hyperparámetros, posteriormente hace un cálculo del MSE y $R^2$ para poder escoger los hiperparámetros más apropiados.

El valor del grid search es una lista de python que contiene un diccionario

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

x_data = df[["horsepower", "curb-weight", "engine-size", "highway-mpg"]]

y_data = df["price"]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.15, random_state=1)


parameters1 = [{"alpha":[0.001, 0.1, 1, 10 , 100, 1000, 10000, 100000],"normalize":[True, False]}]

RR=Ridge()

Grid1= GridSearchCV(RR,parameters1,cv=4)

Grid1.fit(x_data,y_data)


In [ ]:
BestRR = Grid1.best_estimator_
print(BestRR)

Ahora predecimos sobre nuestros datos de prueba

In [ ]:
BestRR.score(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']], y_test)

In [ ]:
scores = Grid1.cv_results_

scores["mean_test_score"]


In [ ]:
y_hat = BestRR.predict(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])

In [ ]:
Title = 'Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution'
DistributionPlot(y_test, yhat, "Actual Values (Train)", "Predicted Values (Train)", Title)